In [1]:
#Pytorch 版本
import torch as t
from  torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import os
import re
import sys
import gc
import time
import copy
import numpy as np
from collections import Counter
from tqdm import tqdm

In [2]:
#read data and preprocess
text = []
for file in os.listdir('./data/Emailtextdata/'):
    with open(os.path.join('./data/Emailtextdata',file),'r',errors='ignore') as f:
        text.extend(f.read().splitlines())
text = [x.replace('*','') for x in text]
text = [re.sub('[^ \fA-Za-z0-9_]', '', x) for x in text]
text =[x for x in text if x !='']
len(text)

1927820

In [3]:
#len(text)
raw_data =[]
for x in text:
    raw_data.extend(x.split(' '))
raw_data =[x for x in raw_data if x !=' ']
del text
gc.collect()

0

In [4]:
vocab = set(raw_data)
vocab_size = len(vocab)
freqs = Counter(raw_data)


In [5]:
#建立索引和建立CBOW窗口
word_to_idx = {word :i for i,word in enumerate(vocab)}
idx_to_word ={i:word for i,word in enumerate(vocab)}
def make_context(context,word_to_idx):
    idx = [word_to_idx[w] for w in context]
    return t.tensor(idx,dtype =t.long)
data =[]
for i in range(2,len(raw_data)-2):
    context = [raw_data[i-2],raw_data[i-1],raw_data[i+1],raw_data[i+2]]
    target = raw_data[i]
    data.append((context,target))
data[:5]
    

[(['Project', 'Gutenbergs', 'of', 'Tom'], 'Etext'),
 (['Gutenbergs', 'Etext', 'Tom', 'Swift'], 'of'),
 (['Etext', 'of', 'Swift', 'And'], 'Tom'),
 (['of', 'Tom', 'And', 'His'], 'Swift'),
 (['Tom', 'Swift', 'His', 'Submarine'], 'And')]

In [6]:
freqs_pow = t.Tensor([freqs[idx_to_word[i]] for i in range(len(vocab))]).pow(0.75)
freqs_pow= freqs_pow/freqs_pow.sum()
del raw_data
gc.collect()

0

In [7]:
#设置GPU

divice = t.device('cuda:0')
def neg_sample(num_samples,label):
    w= np.random.choice(len(freqs_pow),(len(label),num_samples),p=freqs_pow.numpy())
    if label.is_cuda:
        return t.tensor(w).to(divice)
    else:
        return t.tensor(w)


In [8]:
def sigmoid(x):
    return 1 / (1 + t.exp(-x))

我们考虑公式
$-\text{log} \frac{1}{1+\text{exp}\left(-u_c^\top (v_{o_1} + \ldots + v_{o_{2m}}) /(2m)\right)}  - \sum_{k=1, w_k \sim \mathbb{P}(w)}^K \text{log} \frac{1}{1+\text{exp}\left((u_{i_k}^\top (v_{o_1} + \ldots + v_{o_{2m}}) /(2m)\right)}$

In [43]:
#CBOW 进行负采样
class CBOW(nn.Module):
    # 设置 word2vec隐藏层维度，context_size 上下文窗口大小
    def __init__(self,vocab_size,embeding_dim,context_size=2):
        self.embedding_dim = embeding_dim
        self.context_size = context_size
        super(CBOW,self).__init__()
        self.embeddings = nn.Embedding(vocab_size,embeding_dim)
        self.embeddings.weight.data.uniform_(-0.5 / vocab_size, 0.5 / vocab_size)
    def forward(self,inputs,label):
        
        #进行负采样
        
        #现在我们来考虑单个情况 具体链接 请看 http://zh.gluon.ai/chapter_natural-language-processing/approx-training.html 的详解
        #因为我们最后都要对loss取平均 
        """
        negs = neg_sample(5, label)
        u_embeds = self.embeddings(label).view(len(label), -1)
        v_embeds_pos = self.embeddings(inputs).mean(dim=1)
        v_embeds_neg = self.embeddings(negs).mean(dim=1)
        loss1 = t.diag(t.matmul(u_embeds, v_embeds_pos.transpose(0, 1)))
        loss2 = t.diag(t.matmul(u_embeds, v_embeds_neg.transpose(0, 1)))
        loss1 = -t.log(1 / (1 + t.exp(-loss1)))
        loss2 = -t.log(1 / (1 + t.exp(loss2)))
        loss = (loss1.mean() + loss2.mean())
        return loss
        """
        
        negs = neg_sample(5,label)
        u_embed = self.embeddings(label).view(len(label),-1)
        v_embed_target =self.embeddings(inputs).transpose(0,1)# shape =[len(label),context_size*2,embeding_dim]->#shape = =[context_size*2,len(label),embeding_dim]
        #v_embed_target.transpose(0,1)#shape = =[context_size*2,len(label),embeding_dim]
        v_embed_negs = self.embeddings(negs).transpose(0,1)
        
        loss1 = -sum([t.sigmoid((vi*u_embed).sum(dim=1)) for vi in v_embed_target]).sum()
        loss1 =loss1/(len(label)*4)
        
        loss2 = -sum([t.sigmoid(-(vi*u_embed).sum(dim=1)) for vi in v_embed_negs]).sum()
        loss2 =loss2/(len(label))    
        losstotal = loss1+loss2
        return losstotal    
       
        
        

In [46]:
batch_size = 4096
device = t.device('cuda:0')
losses = []
# loss_function = nn.NLLLoss()
model = CBOW(vocab_size,embeding_dim=50)
model.to(device)
# model = torch.nn.DataParallel(model, device_ids=[0, 1]).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [11]:
batch_size = 4096
data_iter = DataLoader(data, batch_size=batch_size,
                                        shuffle=False, num_workers=2)


In [12]:
king = model.embeddings(t.tensor(word_to_idx['king']).cuda())
queen = model.embeddings(t.tensor(word_to_idx['queen']).cuda())
man = model.embeddings(t.tensor(word_to_idx['man']).cuda())
woman = model.embeddings(t.tensor(word_to_idx['woman']).cuda())
((king - man + woman) * queen).sum() / \
    (t.sqrt(((king - man + woman)**2).sum()) * t.sqrt((queen**2).sum()))


tensor(-0.1543, device='cuda:0', grad_fn=<DivBackward0>)

In [47]:
for epoch in range(20):
    total_loss = t.Tensor([0])
    num = 0
    for context, target in tqdm(data_iter):
        num += 1
        context_ids = []
        for i in range(len(context[0])):
            context_ids.append(make_context([context[j][i] for j in range(len(context))], word_to_idx))
        context_ids = t.stack(context_ids)
        context_ids = context_ids.to(device)
        model.zero_grad()
        label = make_context(target, word_to_idx)
        
        label = label.to(device)
        loss = model(context_ids, label)
#         loss = loss_function(log_probs, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)
    print('epoch %d loss %.4f' %(epoch, total_loss / num))
print(losses)

  0%|          | 0/5005 [00:00<?, ?it/s]

  0%|          | 1/5005 [01:46<147:47:40, 106.33s/it]

  0%|          | 2/5005 [01:46<103:30:16, 74.48s/it] 

  0%|          | 4/5005 [01:46<72:26:54, 52.15s/it] 

  0%|          | 6/5005 [01:46<50:43:03, 36.52s/it]

...

 30%|███       | 1507/5005 [03:27<03:15, 17.85it/s]

 30%|███       | 1509/5005 [03:27<03:13, 18.08it/s]

 30%|███       | 1511/5005 [03:27<03:16, 17.76it/s]

 30%|███       | 1513/5005 [03:27<03:17, 17.68it/s]

 30%|███       | 1515/5005 [03:27<03:13, 18.01it/s]

In [13]:
king = model.embeddings(t.tensor(word_to_idx['king']).cuda())
queen = model.embeddings(t.tensor(word_to_idx['queen']).cuda())
man = model.embeddings(t.tensor(word_to_idx['man']).cuda())
woman = model.embeddings(t.tensor(word_to_idx['woman']).cuda())

((king - man + woman) * queen).sum() / \
    (t.sqrt(((king - man + woman)**2).sum()) * t.sqrt((queen**2).sum()))

tensor(-0.2736, device='cuda:0', grad_fn=<DivBackward0>)

In [9]:
#原始softmax版本
class CBOW2(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(len(inputs), -1)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return(log_probs)


tensor([[ 0.3195,  0.4107,  0.3547],
        [-0.4797, -1.3517,  0.6245]])

NameError: name 'model' is not defined

In [26]:
negs = neg_sample(4,t.tensor([1,2,3,0,2]))

In [33]:
d = a.transpose(0,1).shape

In [41]:
b.view(5,-1).shape

torch.Size([5, 10])

In [42]:
loss1 = -sum([t.sigmoid((vi*b).sum(dim=1)) for vi in d]).sum()

In [43]:
loss1

tensor(-14.9708, grad_fn=<NegBackward>)